In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from utils import get_knn_objects, get_sample_1k_objects
from enums import DatasetDirs

from LMI import LMI
from knn_search import evaluate_knn_per_query

26-03-21 12:25 INFO: Loading faiss with AVX2 support.
26-03-21 12:25 INFO: Loading faiss.


In [3]:
from scipy.optimize import linear_sum_assignment
from sklearn.model_selection import train_test_split

from imports import logging

from pathlib import Path
from timeit import default_timer as timer

from utils import get_knn_objects, get_sample_1k_objects
from enums import DatasetDirs, DEFAULT_DESCRIPTORS, KNN_OBJECTS

# BEGIN NEW IMPORTS
from mindex_new import MIndex
from profiset import *
# END

from LMI import LMI
from knn_search import evaluate_knn_per_query

In [6]:
li = LMI(DatasetDirs.COPHIR_1M.value)
df = li.get_dataset(normalize = True)

26-03-21 12:44 INFO: Loaded dataset of shape: (1000000, 285)


In [7]:
df_result = li.train(df, {"GMM": [{'comp': 69, 'cov_type': 'spherical', 'max_iter': 1, 'init_params': 'random'},
                                  {'comp': 69, 'cov_type': 'spherical', 'max_iter': 1, 'init_params': 'random'}]},
                     should_erase = True)

26-03-21 12:44 INFO: Training GMM model with values of shape (1000000, 282): n. of clusters=69 | covariance type=spherical | max iterations=1 | initial parameters=random
26-03-21 12:44 INFO: Training level 1
26-03-21 12:44 INFO: Training GMM model with values of shape (616, 282): n. of clusters=69 | covariance type=spherical | max iterations=1 | initial parameters=random
26-03-21 12:44 INFO: Training GMM model with values of shape (66297, 282): n. of clusters=69 | covariance type=spherical | max iterations=1 | initial parameters=random
26-03-21 12:44 INFO: Training GMM model with values of shape (2, 282): n. of clusters=69 | covariance type=spherical | max iterations=1 | initial parameters=random
26-03-21 12:44 WARNING: Reducing the number of components from 69 to 1 since the number of                           training samples (2) is less than 69
26-03-21 12:44 INFO: Training GMM model with values of shape (202, 282): n. of clusters=1 | covariance type=spherical | max iterations=1 | i

ValueError: Found array with 1 sample(s) (shape=(1, 282)) while a minimum of 2 is required.

In [6]:
df_result.head()["object_id"]

0    234960793
1    333276323
2    376001543
3    190266810
4    386843341
Name: object_id, dtype: int64

In [ ]:
final_results = {}

object_checkpoints = [500, 1000, 3000, 5000, 10000, 50000, 100000, 200000, 300000, 500000, 750000]
checkpoint_count = len(object_checkpoints) 

knns = li.load_knns()
queries = get_sample_1k_objects(df_result, "/storage/brno6/home/tslaninakova/learned-indexes/datasets/queries.data")
query_count = len(queries)

model_times = [0] * checkpoint_count
model_recall = [0] * checkpoint_count

In [33]:
len(queries)

998

In [51]:
search_result = li.search(df_result, queries.iloc[2]["object_id"], stop_cond_objects = object_checkpoints, debug = False)

[{'C.1.3.52': 0.999967660151454}, {'C.1.3.58': 1.3246001152590667e-05}, {'C.1.3.48': 8.799674833490675e-06}, {'C.1.3.19': 8.775182432037667e-06}, {'C.1.3.60': 1.5189901498100226e-06}, {'C.1.3.62': 5.705823146576227e-26}, {'C.1.3.78': 1.542767747273649e-27}, {'C.1.3.10': 6.999481478578385e-32}, {'C.1.3.15': 5.569692635077321e-34}, {'M.1.4': 1.006907796896356e-37}, {'C.1.3.17': 3.284496029211549e-39}, {'C.1.3.2': 2.8861805572995803e-41}, {'C.1.3.1': 8.205019240792991e-45}, {'C.1.3.27': 6.888698688443052e-47}, {'C.1.3.57': 2.2717328646180813e-47}, {'C.1.3.49': 4.0085468514255093e-48}, {'C.1.3.46': 4.0467887414784643e-50}, {'M.1.2': 1.826036427001211e-51}, {'C.1.3.23': 4.795380980279364e-52}, {'C.1.3.53': 9.890674156055637e-53}, {'C.1.3.64': 5.430203048815713e-55}, {'C.1.3.34': 1.860607133627583e-55}, {'C.1.3.45': 5.440791208287042e-58}, {'C.1.3.35': 7.129500171543054e-66}, {'C.1.3.61': 5.979961074751226e-67}, {'C.1.3.13': 4.090984859887018e-67}, {'C.1.3.47': 2.9254595161178857e-68}, {'C.1

[{'C.1.4.85': 2.028469002482696e-207}, {'C.1.3.68': 1.1223538752177123e-208}, {'C.1.4.45': 6.48401992270835e-211}, {'C.1.3.24': 1.2346553372582594e-216}, {'C.1.2.3': 2.715672288220478e-222}, {'M.1.0': 5.328096755849362e-223}, {'C.1.3.26': 3.434638758150322e-227}, {'C.1.2.79': 2.1753140411649054e-227}, {'C.1.3.76': 1.5351979773605345e-230}, {'C.1.4.42': 5.273486636752292e-233}, {'C.1.2.36': 8.067493463313502e-234}, {'C.1.2.80': 2.0225377077422174e-243}, {'C.1.2.46': 9.295315768591024e-244}, {'C.1.2.68': 3.308587160392888e-244}, {'C.1.4.75': 6.895560131372599e-245}, {'C.1.2.50': 1.1628189275278134e-245}, {'C.1.3.85': 2.6320277509033775e-247}, {'C.1.2.2': 1.971607284300128e-248}, {'C.1.2.54': 1.7345645811424616e-248}, {'C.1.4.38': 1.518970684720548e-250}, {'M.1.1': 1.7803324162778295e-253}, {'C.1.2.20': 1.624334973993588e-253}, {'C.1.2.35': 3.04339341668227e-254}, {'C.1.2.84': 5.53064095865554e-256}, {'C.1.4.72': 1.0502038987601469e-257}, {'C.1.3.12': 6.236691657982545e-258}, {'C.1.4.81':

[{'C.1.2.52': 0.0}, {'C.1.2.57': 0.0}, {'C.1.2.62': 0.0}, {'C.1.2.70': 0.0}, {'C.1.2.72': 0.0}, {'C.1.2.73': 0.0}, {'C.1.2.74': 0.0}, {'C.1.2.76': 0.0}, {'C.1.2.81': 0.0}, {'C.1.2.85': 0.0}, {'C.1.0.1': 0.0}, {'C.1.0.4': 0.0}, {'C.1.0.5': 0.0}, {'C.1.0.6': 0.0}, {'C.1.0.7': 0.0}, {'C.1.0.10': 0.0}, {'C.1.0.12': 0.0}, {'C.1.0.15': 0.0}, {'C.1.0.16': 0.0}, {'C.1.0.17': 0.0}, {'C.1.0.18': 0.0}, {'C.1.0.20': 0.0}, {'C.1.0.22': 0.0}, {'C.1.0.26': 0.0}, {'C.1.0.27': 0.0}, {'C.1.0.29': 0.0}, {'C.1.0.30': 0.0}, {'C.1.0.31': 0.0}, {'C.1.0.32': 0.0}, {'C.1.0.33': 0.0}, {'C.1.0.34': 0.0}, {'C.1.0.35': 0.0}, {'C.1.0.36': 0.0}, {'C.1.0.37': 0.0}, {'C.1.0.38': 0.0}, {'C.1.0.42': 0.0}, {'C.1.0.43': 0.0}, {'C.1.0.44': 0.0}, {'C.1.0.45': 0.0}, {'C.1.0.47': 0.0}, {'C.1.0.48': 0.0}, {'C.1.0.49': 0.0}, {'C.1.0.50': 0.0}, {'C.1.0.51': 0.0}, {'C.1.0.52': 0.0}, {'C.1.0.53': 0.0}, {'C.1.0.54': 0.0}, {'C.1.0.55': 0.0}, {'C.1.0.56': 0.0}, {'C.1.0.57': 0.0}, {'C.1.0.58': 0.0}, {'C.1.0.59': 0.0}, {'C.1.0.60': 0.0

In [50]:
queries.iloc[2]["object_id"]

1595410.0

In [37]:
search_result

{'id': 1595410,
 'time_checkpoints': [0.010493040084838867,
  0.010510444641113281,
  0.010523557662963867,
  0.01053762435913086,
  0.010548830032348633,
  0.01286172866821289,
  0.01537322998046875,
  0.015778303146362305,
  0.016162633895874023,
  0.020421743392944336,
  0.020772933959960938],
 'popped_nodes_checkpoints': [['M.1.3', 'C.1.3.52'],
  ['M.1.3', 'C.1.3.52', 'C.1.3.58'],
  ['M.1.3', 'C.1.3.52', 'C.1.3.58', 'C.1.3.48'],
  ['M.1.3', 'C.1.3.52', 'C.1.3.58', 'C.1.3.48', 'C.1.3.19'],
  ['M.1.3', 'C.1.3.52', 'C.1.3.58', 'C.1.3.48', 'C.1.3.19', 'C.1.3.60'],
  ['M.1.3',
   'C.1.3.52',
   'C.1.3.58',
   'C.1.3.48',
   'C.1.3.19',
   'C.1.3.60',
   'C.1.3.62',
   'C.1.3.78',
   'C.1.3.10',
   'C.1.3.15',
   'M.1.4',
   'C.1.4.59',
   'C.1.4.79',
   'C.1.4.31',
   'C.1.4.23',
   'C.1.4.41',
   'C.1.4.55',
   'C.1.4.43',
   'C.1.4.32',
   'C.1.4.69',
   'C.1.4.0',
   'C.1.4.26',
   'C.1.3.17',
   'C.1.4.47',
   'C.1.3.2'],
  ['M.1.3',
   'C.1.3.52',
   'C.1.3.58',
   'C.1.3.48',
   '

In [16]:
from knn_search import gt_knn



ImportError: cannot import name 'gt_knn'

In [15]:
for i in range(query_count):
    search_result = li.search(df_result, queries.iloc[i]["object_id"], stop_cond_objects = object_checkpoints, debug = False)

    time_checkpoints = search_result['time_checkpoints']
    recall = evaluate_knn_per_query(search_result, df_result, knns)

    for j in range(checkpoint_count):
        model_times[j] += time_checkpoints[j]
        model_recall[j] += recall[j]

model_times = list(map(lambda x: x / query_count, model_times))
model_recall = list(map(lambda x: x / query_count, model_recall))

final_results[identifier] = {"model_times": model_times, "model_recall": model_recall}

#with open(f"{self.destination}/{identifier}.csv", 'w') as f:
#    f.write("model_times,model_recall\n")
#    for i in range(checkpoint_count):
#        f.write("%f,%f\n"%(model_times[i], model_recall[i]))

predictions:  [{'value_l': 3, 'votes_perc': 1.0}, {'value_l': 4, 'votes_perc': 2.1603528869506548e-35}, {'value_l': 2, 'votes_perc': 3.9935116214486e-70}, {'value_l': 0, 'votes_perc': 2.337599174179656e-95}, {'value_l': 1, 'votes_perc': 5.189968691158296e-234}]
priority_q:  [{'M.1.3': 1.0}, {'M.1.4': 2.1603528869506548e-35}, {'M.1.2': 3.9935116214486e-70}, {'M.1.0': 2.337599174179656e-95}, {'M.1.1': 5.189968691158296e-234}]
Evaluating k-NN performance on 11 checkpoints: [2674, 5756, 9391, 12384, 15850, 51059, 100902, 201012, 300385, 503129, 752447]


KeyError: '38070170.0'